In [0]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import datetime as dt
import matplotlib.pyplot as plt

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

**Import Raw Data**

In [0]:
df = pd.read_csv('../input/incident_event_logdata.csv',delimiter=',')

#  **Exploratory Data Analysis**

In [0]:
pd.set_option('max_rows', None)
pd.set_option('max_columns', None)
pd.set_option('max_colwidth', None)
nRow, nCol = df.shape
print(f'There are {nRow} rows and {nCol} columns')
df.head()

In [0]:
df.info()

**Data Processing / Data Munging**

In [0]:
# with the initial inspection, there are a lot of ? in data 
df.replace('?', np.NaN, inplace = True)
df.info()

In [0]:
# drop the columns that most values is nan
df1 = df.copy()
df1.drop(columns = ['cmdb_ci','problem_id','rfc','vendor','caused_by'], inplace = True)
# remove impact and urgency, since Priority value is directly computed from them.
df1.drop(columns = ['impact','urgency'], inplace = True)
df1.head()

In [0]:
# extract the numbers from the data 
pattern = r'(\d{1,4})'
colum = ['caller_id','opened_by','sys_created_by','sys_updated_by','location','category','subcategory','u_symptom','priority','assignment_group','assigned_to', 'closed_code', 'resolved_by']

for col in colum:
    df1[col] = df1[col].str.extract(pattern)

# check where the value is missed
#missInd = np.asarray(np.nonzero(pd.isnull(df1.caller_id.values))).flatten()
#missdf = df1[['caller_id','caller_id']].iloc[missInd,:]
df1.head()

In [0]:
df1.info()

# **Data analysis**

**Distribution of columns**

In [0]:
for col in df1.columns:
    print(col, df1[col].nunique())

In [0]:
# Distribution and correlation among features
#%matplotlib notebook
idencolum = ['opened_by','sys_created_by','sys_updated_by','assignment_group','assigned_to','resolved_by']
timeColum = ['opened_at', 'sys_created_at','sys_updated_at','resolved_at','closed_at']
df_identify = df1[idencolum]
for col in idencolum:
    df_identify[col] = pd.to_numeric(df_identify[col], errors='coerce').fillna(0).astype(np.int64)
plt.figure()
pd.plotting.scatter_matrix(df_identify,figsize=(15,15))
plt.savefig(r"Distribution and correlation among features.png")

From the plot of scatter matrix, it can be observed that:
1. Identifier of the user who resolved the incident (resolved_by) is relative-uniformly distributed while the other features are inbalanced. 
2. It is clearly indicated that there is a linear correlation between 'opened_by' and 'sys_created_by'.
3. A vague linear line is observed in the plot between 'assigned_to' and 'resolved_by', inferring that the correlation between these two features: some CI which was initially assigned to the incident is actually the one can reslove the incident.

In [0]:
# continue 
othercolum = ['reassignment_count','reopen_count','made_sla','category','priority','closed_code']

df_other = df1[othercolum]
for col in othercolum:
    df_other[col] = pd.to_numeric(df_other[col], errors='coerce').fillna(0).astype(np.int64)
plt.figure()
pd.plotting.scatter_matrix(df_other,figsize=(15,15))
plt.savefig(r"Distribution and correlation among features_2.png")

Comments based on this scatter matrix plot:
1. the most of reassignment_count and reopen_count is 0. 
2. about 90% instance have SLA (service level agreement).
3. Most cases are in priority 3, i.e. moderate. Therefore, the selection of the proper metrics is important for predicting the priority of incidents.
4. the dominated closed_code is about 6-10. 
5. Data is significatnly inbalanced, so need normalize in the analysis.

**Specific Features Understanding**

In [0]:
# incident_state
plt.figure()
bins = np.arange(0,df1.incident_state.nunique()+2,1)

ax = df1.incident_state.hist(width =0.6,bins= bins,figsize=(6,4),align='mid')
plt.xticks(rotation=45)
ax.grid(False)
ax.set_xticks(bins[:-1])
ax.set_ylabel('Numbers')
ax.set_title('Distribution of the incident_state')

From the figure, the incident can be in 7 different states as given time before it get closed. The number of 'New' and 'Active' is larger than the 'closed' case is because that the instance can be reopened in case incident resolution was rejected by the caller and in the data set there is a field i.e. reopen_count that captures the number of times got reopened.

In [0]:
# relationship between made_sla and reopen_count
sla = (df1[(df1.made_sla == True) & (df1.reopen_count>0)].groupby('number')['reopen_count'].mean()).mean()
nosla = (df1[(df1.made_sla == False) & (df1.reopen_count>0)].groupby('number')['reopen_count'].mean()).mean()
print(f'mean reopen_count for having SLA {sla} and without SLA {nosla}')

In [0]:
# Distribution of closed_code; relationship between close_code and reopen_count
import seaborn as sns

fig, (ax1, ax2) = plt.subplots(nrows=1, ncols=2,figsize=(12,4))
bins=np.arange(0,df1.closed_code.nunique()+2,1)
df1[df1.incident_state=='Closed'].sort_values('closed_code').closed_code.hist(width =0.8,bins = bins,align='left',ax=ax1)
ax1.grid(False)
ax1.set_xticks(bins[:-1])
ax1.set_ylabel('Numbers')
ax1.set_title('Distribution of closed_code')


dfclosecode = df1[(df1.reopen_count>0) & (df1.incident_state=='Closed')]
dfclose_reopen = dfclosecode.groupby('closed_code').reopen_count.mean()
dfclose_reopen.plot.bar(ax=ax2)
ax2.grid(False)
ax2.set_ylabel('mean of Reopen_count')
ax2.set_xticks(bins[:-1])
ax2.set_title('closed_code vs. reopen_count')

plt.show()

From this figure, it can be observed that:
1. the dominated closed_code is 6
2. the reopen_count is 0 for closed_code = 12,13,14,15.
3. the mean reopen_count is large for closed_code 10, which means the instance resolution is easily rejected for this closed_code.

In [0]:
# assigned to and resolved by 
df_ar = df1[['assigned_to','resolved_by']]
df_ar['equal'] = np.where(df_ar.assigned_to == df_ar.resolved_by,1,0)
equal_num = df_ar['equal'].sum()
print(equal_num/df_ar.shape[0] * 100)

Only 0.4% of CI which was initially assigned to the incident is actually the one can reslove the incident. Hence, an accurate prediction of 'resolved_by' can significantly improve the efficiency of the incidence management process.

In [0]:
# completion time for incident resolution 
from datetime import datetime, date

df_closed = df1[df1.incident_state=='Closed'].reset_index()
df_closed.opened_at =  pd.to_datetime(df_closed.opened_at, format='%d/%m/%Y %H:%M')
df_closed.closed_at =  pd.to_datetime(df_closed.closed_at, format='%d/%m/%Y %H:%M')
df_closed['completion_time_hours'] = (df_closed.closed_at- df_closed.opened_at).dt.total_seconds()/3600
print(f'The mean of completion time for incident resolution is {df_closed.completion_time_hours.mean()}')

#plots
plt.figure()
ax = df_closed['completion_time_hours'].plot(figsize=(10,4))
ax.grid(False)
ax.set_ylabel('completion time in hours')
ax.set_title('Distribution of completion_time')

In [0]:
# completion time vs closed code; completion time vs made_sla

plt.figure()
fig, (ax1, ax2) = plt.subplots(nrows=1, ncols=2,figsize=(12,4))
df_closecode_time = df_closed.groupby('closed_code')['completion_time_hours'].mean()
df_closecode_time.plot.bar(ax=ax1)
ax1.grid(False)
ax1.set_ylabel(' mean completion time in hours')
ax1.set_title('completion time vs closed code')

df_made_sla_time = df_closed.groupby('made_sla')['completion_time_hours'].mean()
df_made_sla_time.plot.bar(ax=ax2)
ax2.grid(False)
ax2.set_ylabel('mean completion time in hours')
ax2.set_title('completion time vs made_sla')
plt.show()

This figure indicates that:
1. For closed_code of 13, the mean completion time is about 1000 hours, which is relative large compared to the others. This observation is unexpected since its reopen_count is 0 based on previous plot. 
2. The mean completion time for the instances without SLA is much longer than those having SLA. Therefore, making SLA would be helpful for enhancing the efficiency of incident management process.